This notebook preprocesses the ERA5 data for PAMTRA. 
As a second step, this notebook forward-simulates the brightness temperatures for the HAMP radiometer channels (Mech et al., 2014, https://doi.org/10.5194/amt-7-4539-2014),
using the ERA5 training data.

In [1]:
import pyPamtra
import datetime
import numpy as np
import pickle
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr
get_ipython().run_line_magic('matplotlib', 'inline')

import os
import sys
# import regression packages
current_path=os.getcwd()
sys.path.insert(1,current_path+"/../src/")
sys.path.insert(2,current_path+"/../retrieval")
import ERA5_Tb               ##---> needed to work with PAMTRA simulated TBs
import Regression_Retrieval  ##---> class for executing the regression retrieval

In [2]:
# this creates the input of ERA5 for handling in the readERA5 module
from cdo import *

In [3]:
data_path="/work/bb1086/pamtra_hamp_retrieval/"

## Switches

Defining if new simulations shall be conducted and which training data should be used.

In [4]:
run_simulation=True
take_random_spring=False
take_synth_ar_dates=True
rerun_dates=True

## ERA5 Preprocessing

Initialising the ERA5 preprocessing class for later use in PAMTRA

In [7]:
ERA5_preprocess=ERA5_Tb.ERA5_Preprocessing

## Run PAMTRA

Initialising the PAMTRA_Handler class that is the linkage between preprocessed ERA5 and PAMTRA

In [8]:
PAMTRAhandler=ERA5_Tb.PAMTRA_Handler

## PAMTRA Output Plots

In [9]:
# Some quicklook PAMTRA illustrations, not further used in the routine, but useful for quick data analysis.
def plotDataHyd(lon,lat,data):
    data[data < 0.05] = np.nan
    
    map_proj=ccrs.Mollweide(central_longitude=-30)
    data_proj=ccrs.PlateCarree()
    ax = plt.subplot(221,projection=map_proj)
    ax.coastlines()
    plt.pcolormesh(lon,lat,data[:,:,0],transform=data_proj,cmap='jet')
    plt.colorbar()
    ax = plt.subplot(222,projection=map_proj)
    ax.coastlines()
    plt.pcolormesh(lon,lat,data[:,:,1],transform=data_proj,cmap='jet')
    plt.colorbar()
    ax = plt.subplot(223,projection=map_proj)
    ax.coastlines()
    plt.pcolormesh(lon,lat,data[:,:,2],transform=data_proj,cmap='jet')
    plt.colorbar()
    ax = plt.subplot(224,projection=map_proj)
    ax.coastlines()
    plt.pcolormesh(lon,lat,data[:,:,3],transform=data_proj,cmap='jet')
    plt.colorbar()
    plt.show()
    return None

def plotMap(lon,lat,data):
    proj = ccrs.NorthPolarStereo(central_longitude=10)
    data_crs = ccrs.PlateCarree()
    ax = plt.axes(projection=proj)
    ax.coastlines()
    plt.pcolormesh(lon,lat,data[:,:],transform=data_crs,cmap='jet')
    plt.colorbar()

    plt.show()

    return


### Training data day generation

In [11]:
def create_random_dates(old_dates,number_of_dates=3):
    """
    This routine defines random dates from spring season (March, April) out of the year range from 1979 to 2021.

    Input:

    old_dates: list
        Dates (as strings) that are already defined. They will be checked for not being repeated

    number_of_dates: int
        Number of dates that should be chosen randomnly. Default is 3. Mostly 50 are used.

    Returns:
    
    dates: list
        Dates to consider as a list of strings.
    """
    import random
    dates=[]
    d=0
    while d <number_of_dates:
        year=random.randint(1979,2021)
        month=random.randint(3,4)
        if month==3:
            day=random.randint(1,31)
        else:
            day=random.randint(1,30)    
        
        date=str(year*10000+month*100+day)
        if not date in old_dates:
            dates.append(date)
            d+=1
    return dates

def get_dates(specific_dates=[],rerun_dates=True,take_random_dates=True,take_synth_ar_dates=False,
             no_of_dates=5):
    """
    This routine gets all the dates. 
    
    Input:
        specific_dates: list
            List of specific dates to be used in any way. Default is empty list.
        rerun_dates: boolean
            if yes, new dates are used, else old ones are used using PAMTRASIM_analysis
    """
    if len(specific_dates)!=0:
        if take_random_dates:
            if rerun_dates:
                #Add randomn dates to specific dates
                create_random_dates(specific_dates,number_of_dates=no_of_dates)
            else:
                import PAMTRA_sim_analysis
                PAMTRASIM_analysis=PAMTRA_sim_analysis.PAMTRASIM_analysis
                all_available_days=PAMTRASIM_analysis.list_all_simulated_days(
                    data_path="/work/bb1086/pamtra_hamp_retrieval/",hour=hour_to_analyse)
        if take_synth_ar_dates:
            dates=["20110317","20110423","20150314","20160311","20180224","20180225",
                       "20190319","20200416","20200419"]
        else:
            # Use just the specific dates
            dates=specific_dates
    else:
        # Default training dataset.
        dates=['19790312', '19810330', '19810424', '19820320', '19820416','19830316', 
               '19830331', '19830414', '19840308', '19840413','19840428', '19880330', 
               '19890422', '19900409', '19900411', '19900418', '19910301', '19910428', 
               '19920302','19920413', '19930303', '19930430','19940421', '19950306', 
               '19950317', '19970322', '19980324', '19980409', '20030415', '20050401',
               '20060317', '20070324','20080313', '20080404', '20080411', '20080430', 
               '20090324', '20100323', '20110311', '20110329', '20110413', '20120312',
               '20130313','20140309', '20140330', '20160409', '20180319', '20180326', 
               '20200326','20210402', '20220310']
    return dates        

## Run Main Routine

In [13]:
hour_to_analyse="10" # UTC
specific_dates=["20150314"]
dates=get_dates(specific_dates=specific_dates,take_random_dates=False,take_synth_ar_dates=take_synth_ar_dates)
print("Training data:", sorted(dates))

Training data: ['20110317', '20110423', '20150314', '20160311', '20180224', '20180225', '20190319', '20200416', '20200419']


Loop over all days and preprocess the ERA5 data that is then implemented into PAMTRA by the PAMTRA handler. 
If the corresponding files are already there, they are simply loaded.

In [ ]:
from datetime import datetime
if run_simulation:
    import ERA5_Tb
    default_area=[-30,50,65,89]#[5,10,65,70]
    for date in sorted(dates):
        print(date)
        yyyy = int(date[0:4])
        mm   = int(date[4:6])
        dd   = int(date[6:8])
        now = datetime.now()

        current_time = now.strftime("%H:%M:%S")
        print("Current Time =", current_time)

        era5_processing_cls=ERA5_preprocess(yyyy,mm,dd,
                        '/home/b/b380702/pamtra/descriptorfiles/descriptor_file_ecmwf.txt',
                        outPath='/scratch/u/u300737/',area=default_area,timestep=int(hour_to_analyse)+1)
        temporary_pamtrahandler=PAMTRAhandler(era5_processing_cls)
        ## ---> which output path      
        era5_processing_cls.runCDO()
        print("CDO done")
        era5_existent,pamtra_existent=era5_processing_cls.checkfor_era5_pamtra_files()
        if not era5_existent:
            era5_processing_cls.readERA5(inPath='/scratch/u/u300737/',step=4,cut_levels=5)
            era5_processing_cls.create_pamData_dict(step=4)
            print("entire ERA5 read")
        else:
            print("Processed ERA5 already created")
            
        pamtrahandler=PAMTRAhandler(era5_processing_cls)
        if not pamtra_existent:
            print("PAMTRA TBs not there already, they should be calculated")
            if run_simulation:
                # reduce to just ocean grid points
                filter = np.empty(era5_processing_cls.pam._shape2D,dtype=bool)
                filter[:,:] = False
                filter[era5_processing_cls.pam.p['sfc_type'] == 0] = True
                era5_processing_cls.pam.filterProfiles(filter)
                pamtrahandler.runPAMTRA()
                pamtrahandler.collectERA5()
                pamtrahandler.reducePAMTRAResults(instrument='hamp')
        else:
            print("PAMTRA TBs already calculated")

20110317
Current Time = 15:19:36
Version of 2023-05-16 runned
File to check: /scratch/u/u300737/reduced_ml_20110317_10_130.nc
All sf files now calculated
all IV files calculated
CDO done
entire ERA5 read
PAMTRA TBs not there already, they should be calculated
passive
OUTPUT path: /scratch/u/u300737/


/home/u/u300737/lib/python/pyPamtra/core.py:838: Warning: sfc_salinity set to 33.0
  warnings.warn("%s set to %s"%(environment,preset,), Warning)
/home/u/u300737/lib/python/pyPamtra/core.py:871: Warning: obs_height set to [833000.0, 0.0]
  warnings.warn("%s set to %s"%(environment,preset,), Warning)
/home/u/u300737/lib/python/pyPamtra/core.py:881: Warning: hydro_reff set to 0
  warnings.warn(qValue + " set to 0", Warning)
/home/u/u300737/lib/python/pyPamtra/core.py:881: Warning: hydro_n set to 0
  warnings.warn(qValue + " set to 0", Warning)
/home/u/u300737/lib/python/pyPamtra/core.py:892: Warning: airturb set to nan
  warnings.warn(qValue + " set to nan", Warning)
/home/u/u300737/lib/python/pyPamtra/core.py:892: Warning: wind_w set to nan
  warnings.warn(qValue + " set to nan", Warning)
/home/u/u300737/lib/python/pyPamtra/core.py:892: Warning: wind_uv set to nan
  warnings.warn(qValue + " set to nan", Warning)
/home/u/u300737/lib/python/pyPamtra/core.py:892: Warning: turb_edr set to n

#### move all pamtra hamp files to another directory for long-term storage #work

In [ ]:
import subprocess
import glob
old_pamtra_outp_path="/scratch/u/u300737/"
old_file_list=glob.glob(old_pamtra_outp_path+"pamtra_hamp_*")
new_pamtra_outp_path=data_path
if not os.path.exists(new_pamtra_outp_path):
    os.makedirs(new_pamtra_outp_path)
for file in old_file_list:
    file_name=file.split("/")[-1]
    status = subprocess.call('cp '+file+" "+new_pamtra_outp_path+file_name, shell=True) 

In [ ]:
# Show last PAMTRA file
pamtra_ds=xr.open_dataset(new_pamtra_outp_path+file_name)
pamtra_ds

#### Move all ERA5 hamp files to another directory #work

In [ ]:
old_era5_outp_path="/scratch/u/u300737/"#pamtra_hamp_20200326_12.nc
old_file_list=glob.glob(old_pamtra_outp_path+"era5_*")
new_era5_outp_path=data_path
if not os.path.exists(new_era5_outp_path):
    os.makedirs(new_era5_outp_path)
for file in old_file_list:
    file_name=file.split("/")[-1]
    #print(file_name)
    status = subprocess.call('cp '+file+" "+new_era5_outp_path+file_name, shell=True) 

In [ ]:
# show last ERA5 file
era5_ds=xr.open_dataset(new_pamtra_outp_path+"era5_"+date+"_"+hour_to_analyse+"_atmos.nc")